Assignment follow the that keypoints
-SVM,Decision Tree,Random forest and Logistic Regression
- Train-test split model training and evaluation
- Cross Validation
- Select best performiing model using CV
- fine tune that model
- Evaluate again
- Apply ada boost on fine tune model

# Import Libraries

In [41]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
import warnings
warnings.filterwarnings('ignore')

## Load dataset

In [2]:
df=pd.read_csv('DataSet/titanic.csv')

In [3]:
df.head() # show top 5 rows of dataset

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df.shape # checck the shape of data

(891, 12)

In [5]:
df.info() # Check the data type of individual columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [6]:
df.describe().T # overview of data

,count,mean,std,min,25%,50%,75%,max
PassengerId,891.0,446.000000,257.353842,1.00,223.5000,446.0000,668.5,891.0000
Survived,891.0,0.383838,0.486592,0.00,0.0000,0.0000,1.0,1.0000
Pclass,891.0,2.308642,0.836071,1.00,2.0000,3.0000,3.0,3.0000
Age,714.0,29.699118,14.526497,0.42,20.1250,28.0000,38.0,80.0000
SibSp,891.0,0.523008,1.102743,0.00,0.0000,0.0000,1.0,8.0000
Parch,891.0,0.381594,0.806057,0.00,0.0000,0.0000,0.0,6.0000
Fare,891.0,32.204208,49.693429,0.00,7.9104,14.4542,31.0,512.3292


In [7]:
df.isnull().sum() # Check the missing value in dataset

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [8]:
df.isnull().sum().sort_values(ascending=False)/len(df)*100

Cabin          77.104377
Age            19.865320
Embarked        0.224467
PassengerId     0.000000
Survived        0.000000
Pclass          0.000000
Name            0.000000
Sex             0.000000
SibSp           0.000000
Parch           0.000000
Ticket          0.000000
Fare            0.000000
dtype: float64

**Observation**
- Cabin contains almost 77% missing value
- Age contains almost 19% missing value


In [9]:
df['Initial']=0
for i in df:
    df['Initial']=df.Name.str.extract('([A-Za-z]+)\.') #lets extract the Salutations

In [10]:
df['Initial'].replace(
    ['Mlle','Mme','Ms','Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don'],
    ['Miss','Miss','Miss','Mr','Mr','Mrs','Mrs','Other','Other','Other','Mr','Mr','Mr'],inplace=True)

In [11]:
#Assigning the missing value
df.loc[(df.Age.isnull())&(df.Initial=='Mr'),'Age']=33
df.loc[(df.Age.isnull())&(df.Initial=='Mrs'),'Age']=36
df.loc[(df.Age.isnull())&(df.Initial=='Master'),'Age']=5
df.loc[(df.Age.isnull())&(df.Initial=='Miss'),'Age']=22
df.loc[(df.Age.isnull())&(df.Initial=='Other'),'Age']=46

In [12]:
df.Age.isnull().sum()# check the missing value

0

In [13]:
df['Cabin'] = df['Cabin'].apply(lambda x:not isinstance(x, float))

In [14]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'Initial'],
      dtype='object')

### Drop Columns

In [15]:
df.drop(['PassengerId','Name','Ticket'],axis=1,inplace=True) #drop the unvaluable columns

In [16]:
df.head(2) # check how many columns are left

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Initial
0,0,3,male,22.0,1,0,7.2500,False,S,Mr
1,1,1,female,38.0,1,0,71.2833,True,C,Mrs


### Label Encoding

In [17]:
label_encoder={}
for x in ['Sex','Embarked','Initial']:
    label_encoder[x]=LabelEncoder()
    df[x]=label_encoder[x].fit_transform(df[x])

In [18]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Initial
0,0,3,1,22.0,1,0,7.2500,False,2,2
1,1,1,0,38.0,1,0,71.2833,True,0,3
2,1,3,0,26.0,0,0,7.9250,False,2,1
3,1,1,0,35.0,1,0,53.1000,True,2,3
4,0,3,1,35.0,0,0,8.0500,False,2,2


**Observation**
- label encoding

## split the feature and target output

In [19]:
x=df[df.columns[1:]].values
y=df.Survived

## Using train-test split

In [20]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42,shuffle=True)

## Logistic Regression Model

In [21]:
lr=LogisticRegressionCV()
lr.fit(X_train,y_train)

LogisticRegressionCV()

In [22]:
y_pred=lr.predict(X_test)

In [30]:
print('Accuracy:',accuracy_score(y_test,y_pred))
print("Precision:",precision_score(y_test,y_pred))
print("Recall_Score:",recall_score(y_test,y_pred))
print("f1_score:",f1_score(y_test,y_pred))

Accuracy: 0.8161434977578476
Precision: 0.7857142857142857
Recall_Score: 0.7415730337078652
f1_score: 0.7630057803468209


## Decision Tree Model

In [36]:
dt=DecisionTreeClassifier()

In [37]:
dt.fit(X_train,y_train)

DecisionTreeClassifier()

In [38]:
y_pred=dt.predict(X_test)

In [40]:
print('Accuracy:',accuracy_score(y_test,y_pred))
print("Precision:",precision_score(y_test,y_pred))
print("Recall_Score:",recall_score(y_test,y_pred))
print("f1_score:",f1_score(y_test,y_pred))

Accuracy: 0.7354260089686099
Precision: 0.6595744680851063
Recall_Score: 0.6966292134831461
f1_score: 0.6775956284153006


## SVM Model

In [43]:
svm=SVC()
svm.fit(X_train,y_train)

SVC()

In [44]:
y_pred=svm.predict(X_test)

In [45]:
print('Accuracy:',accuracy_score(y_test,y_pred))
print("Precision:",precision_score(y_test,y_pred))
print("Recall_Score:",recall_score(y_test,y_pred))
print("f1_score:",f1_score(y_test,y_pred))

Accuracy: 0.6636771300448431
Precision: 0.71875
Recall_Score: 0.25842696629213485
f1_score: 0.3801652892561984
